<a href="https://colab.research.google.com/github/institutohumai/cursos-python/blob/master/Scraping/2_HTTP_Avanzado/scraping_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab" data-canonical-src="https://colab.research.google.com/assets/colab-badge.svg"></a>


# Práctica: Scraping Avanzado 2:

- ChromeDevTools
- Cookies
- Crawling
- Regex, XPath


## Pyautogui, Selenium, Requests

Ahora que vimos `requests` podemos entender que es una manera protocolar de conseguir un recurso en lugar de interactuar a través del sistema operativo con mousse y teclado como con `pyautogui`.

La web está hecha con la tríada de lenguajes: HTML, JS, y CSS. Javascript modifica el contenido dinámicamente, ejecutando código del lado del cliente. Requests no ejecuta el código javascript, por lo que en ese caso es necesario usar una alternativa que sí lo haga. Desde "alto nivel" hacia "bajo nivel" sería: automatizando la interacción con el navegador desde el sistema operativo (pyautogui), automatizando el navegador (selenium), o encontrando una **API oculta o visible para usar requests.**.

Cuando una compañia desarrolla un sitio web muchas veces separa lo que se dice el _frontend_, que es la parte visible del sitio y la cual se ejecuta en tu navegador, del _backend_, la parte del sitio que realiza el computo mas pesado y se ejecuta en servidores/computadoras de la compañia.

Para comunicar el _backend_ con el _frontend_ una forma popular es desarrollar REST APIs, a veces estas son públicas pero a veces están ocultas y las utilizamos sin darnos cuenta cuando interactuamos con un sitio web.

Ejemplo de un sitio web y su API: http://numbersapi.com/

# Ejemplos

#### Modo 1: pyautogui

Lo queremos evitar, es frágil y _ad hoc_ (**y no anda en Colab!**)

In [ ]:
!pip install pyautogui
import pyautogui
from time import sleep

In [153]:
import requests as rq
url = 'https://www.lanacion.com.ar/'

In [ ]:
pyautogui.hotkey('ctrl', 't')
sleep(2)
pyautogui.write(url.split())
sleep(2)
pyautogui.press('enter')

Modo 2:

Emulando un navegador con Selenium

In [ ]:
# Esta celda instala selenium y chromedriver en Colab
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver/usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

In [ ]:
url

'https://www.lanacion.com.ar/'

In [ ]:
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox') # necesarios en colab 
chrome_options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome(options=chrome_options)

In [ ]:
# *vamos* a un sitio
driver.get(url)

In [ ]:
# el driver ahora "está" en ese sitio
driver.page_source[:100]

In [ ]:
# Ejercicio: usar xpath para encontrar los títulos
patron = '' # completar
titulos = [e.text for e in driver.find_elements_by_xpath(patron)]

In [ ]:
titulos

['Reuniones sin acuerdo. Tironeos, negociaciones y un negocio millonario detrás de la falta de gasoil',
 'Disputa. La Nación y la Ciudad volvieron a reunirse por la coparticipación: hablan de un “principio de avance”',
 'Tregua. Fernández almorzó con Pablo Moyano y dieron una señal de unidad en medio de la interna oficialista',
 'A las piñas por Gasoil. Un camionero empezó una brutal pelea con un playero por el cupo al combustible',
 '¿Secuestro en Puerto Madero? Un policía retirado mató a un ladrón que tenía encerrada en el baúl a una víctima',
 '"Fútbol parásito". Las despiadadas críticas para el estilo de Simeone y la respuesta de Koke para Guardiola',
 'Temas del día:']

Tercer modo: requests


In [183]:
import requests 
# para pasar del string con el html a un DOM y usar xpath
from lxml import html
# alternativa 
from bs4 import BeautifulSoup

In [202]:
r = requests.get('https://www.lanacion.com.ar/')

In [185]:
soup = BeautifulSoup(r.content, 'html.parser')

Ahora parseo: 
- BeautifulSoup: funciones importantes son .find(), .find_all(), .get()
- XPATH: un mini lenguaje para recorrer archivos en formato [XML](https://www.w3schools.com/xml/xpath_intro.asp)

In [211]:
[e for e in soup.find_all('h2')]

[<h2 class="com-subhead --twoxs"><a href="/sociedad/provincia-desde-manana-dejara-de-ser-obligatorio-el-uso-de-tapabocas-en-escuelas-trabajos-y-espacios-nid06042022/" title="Así lo anunció el ministro de Salud bonaerense, Nicolás Kreplak, que indicó que ellos continúan recomendando el uso; en el transporte público seguirá siendo obligatorio">Así lo anunció el ministro de Salud bonaerense, Nicolás Kreplak, que indicó que ellos continúan recomendando el uso; en el transporte público seguirá siendo obligatorio</a></h2>,
 <h2 class="com-title --xs"><a class="com-link" href="/politica/una-vecina-de-recoleta-indignada-por-la-muerte-de-su-padre-por-covid-es-buscada-por-pagar-los-nid06042022/" title='"Asesina". Una vecina de Recoleta indignada por la muerte de su padre es buscada por pagar los afiches contra Cristina'><em class="com-lead">"Asesina". </em>Una vecina de Recoleta indignada por la muerte de su padre es buscada por pagar los afiches contra Cristina</a></h2>,
 <h2 class="com-title -

In [212]:
dom = html.fromstring(r.content)

# con XPATH
[''.join(e.itertext()) for e in dom.xpath('//h2')]

['Así lo anunció el ministro de Salud bonaerense, Nicolás Kreplak, que indicó que ellos continúan recomendando el uso; en el transporte público seguirá siendo obligatorio',
 '"Asesina". Una vecina de Recoleta indignada por la muerte de su padre es buscada por pagar los afiches contra Cristina',
 '"Casi se queda afuera". Otra noche mágica en la Champions de Karim Benzema, el delantero que deslumbra al mundo',
 'Reuniones sin acuerdo. Tironeos y un negocio millonario detrás de la falta de gasoil',
 '"Un bolso chiquito". Juzgan al exjefe del Ejército de Cristina Kirchner por enriquecimiento ilícito',
 'Viajes. Las aerolíneas que ya no exigen el uso de barbijo en los vuelos']

In [237]:
# Cualquier elemento cuyo atributo "class" contenga "title"
patron = '//*[contains(@class, "title")]'
[''.join(e.itertext()) for e in dom.xpath(patron)]

['Desde mañana. El uso de tapabocas en escuelas dejará de ser obligatorio en la provincia',
 '"Asesina". Una vecina de Recoleta indignada por la muerte de su padre es buscada por pagar los afiches contra Cristina',
 '"Casi se queda afuera". Otra noche mágica en la Champions de Karim Benzema, el delantero que deslumbra al mundo',
 'Reuniones sin acuerdo. Tironeos y un negocio millonario detrás de la falta de gasoil',
 '"Un bolso chiquito". Juzgan al exjefe del Ejército de Cristina Kirchner por enriquecimiento ilícito',
 'Viajes. Las aerolíneas que ya no exigen el uso de barbijo en los vuelos']

Buenas puntas:

- sitemap
- robots.txt

¿Cómo encuentro el sitemap? Ejemplo de "Google Hacking": uso de operadores en google para mejores búsquedas.

`inurl:sitemap site:lanacion.com.ar` 

### ChromeDevTools

El navegador de Chrome nos ofrece herramientas para analizar elementos de la web. Para eso, vamos a abrir las "Herramientas de Desarrollador" en el menú o con _CTRL + SHIFT + I_.

Ahí hay dos pestañas que van a ser de nuestro interés:

En **Elements** vamos a poder inspeccionar el código HTML, para ubicar los datos de nuestro interés. Para identificar la ubicación de uno, podemos posicionarnos con el cursor sobre el sitio, hacer click derecho, y seleccionar "Inspeccionar elemento".

En la solapa **Network** podemos ver todos los paquetes HTTP que realiza nuestro navegador interactuando con un sitio. Identificando los paquetes de las APIs que traen los datos, podemos _scrapear_ datos más facilmente.

Posibles estrategias:
- Tomar una parte de la info que buscamos del sitio y ponerla en el buscador en _Network_
- Buscar en el HTML dónde se ubica la info que necesitamos
- Buscar en HTML o con DevTools las llamadas de javascript

<img src='https://i2.wp.com/abodeqa.com/wp-content/uploads/2019/02/Inspect-Element-Using-Select-Tool.gif'>


**Ejemplo Bytes:**

Base de Datos de Comercio Exterior del Instituto Nacional de Estadística de Bolivia

Vamos a ingresar al siguiente sitio web:

http://web2.ine.gob.bo:8081/IneComex/BasesComex.aspx

Pasos:

- Ingresar al sitio en una nueva pestaña
- Abrir la pestaña "Network" en las herramientas de desarrollador del navegador
- Llenar el formulario y descargar la base de datos
- Observar el paquete HTTP que realizó el pedido

In [ ]:
!wget http://web2.ine.gob.bo:8081/IneComex/BD/exp2020.rar
!unrar x exp2020.rar ./

--2022-04-06 21:53:55--  http://web2.ine.gob.bo:8081/IneComex/BD/exp2020.rar
Resolving web2.ine.gob.bo (web2.ine.gob.bo)... 200.105.137.5
Connecting to web2.ine.gob.bo (web2.ine.gob.bo)|200.105.137.5|:8081... connected.
HTTP request sent, awaiting response... 200 OK
Length: 161096 (157K) [application/octet-stream]
Saving to: ‘exp2020.rar’

exp2020.rar         100%[===================>] 157.32K   306KB/s    in 0.5s    

2022-04-06 21:53:57 (306 KB/s) - ‘exp2020.rar’ saved [161096/161096]


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from exp2020.rar

Extracting  ./exp2020.txt                                                 20% 40% 60% 81% 99%  OK 
All OK


Podríamos bajarlo directo con Python. Las descargas de distintos archivos de formatos distintos a texto (.txt, .csv, .json...) son iguales, escribiendo a disco en modo "write bytes" ('wb')

In [238]:
# Ejemplo bytes
url = 'https://upload.wikimedia.org/wikipedia/commons/thumb/e/e4/HTML_logo.png/250px-HTML_logo.png'
r = requests.get(url)

In [239]:
with open('imagen.png', 'wb') as out:
  out.write(r.content)

In [ ]:
# vemos la primera parte de la base de datos
!head exp2020.txt 

GESTION|MES|Departamento|NANDINA|des_nandina|des_Pais|des_Medio_Sal|des_Via_Sal|CUCIR3|GCE|CIIU3|CODACT|TNT|Peso_Bruto_Kg|Peso_Neto_Kg|Valor_FOB_Sus
2020.0|7.0|2.0|2106906100| PREPARACIONES ALIMENTICIAS MEJORADORES DE PANIFICACION A BASE DE ESTEVIA|ZONA FRANCA DE BOLIVIA|CARRETERA|ZONAS FRANCAS|0989|122|1549|D6|34.0|53|50|4023
2020.0|10.0|2.0|2106906100| PREPARACIONES ALIMENTICIAS MEJORADORES DE PANIFICACION A BASE DE ESTEVIA|ZONA FRANCA DE BOLIVIA|CARRETERA|ZONAS FRANCAS|0989|122|1549|D6|34.0|53|50|4023
2020.0|6.0|2.0|2208500000|"GIN" Y GINEBRA|COSTA RICA|AEREA|AEREA|1124|122|1551|D3|13.0|70|34|272
2020.0|1.0|2.0|2208500000|"GIN" Y GINEBRA|PERU|CARRETERA|DESAGUADERO|1124|122|1551|D3|13.0|11230|6300|42750
2020.0|1.0|2.0|2208500000|"GIN" Y GINEBRA|CHILE|CARRETERA|ARICA - CHARANA - TAMBO QUEMADO|1124|122|1551|D3|13.0|4500|2520|15480
2020.0|6.0|2.0|2208500000|"GIN" Y GINEBRA|COSTA RICA|CARRETERA|ARICA - CHARANA - TAMBO QUEMADO|1124|122|1551|D3|13.0|755|412|3334
2020.0|7.0|2.0|2208500000|"

In [ ]:
# Lo podemos cargar con pandas para análisis de datos

import pandas as pd
df = pd.read_csv('exp2020.txt', sep='|', encoding="ISO-8859-1")
df.head()

,GESTION,MES,Departamento,NANDINA,des_nandina,des_Pais,des_Medio_Sal,des_Via_Sal,CUCIR3,GCE,CIIU3,CODACT,TNT,Peso_Bruto_Kg,Peso_Neto_Kg,Valor_FOB_Sus
0,2020.0,7.0,2.0,2106906100,PREPARACIONES ALIMENTICIAS MEJORADORES DE PAN...,ZONA FRANCA DE BOLIVIA,CARRETERA,ZONAS FRANCAS,989,122,1549,D6,34.0,53,50,4023
1,2020.0,10.0,2.0,2106906100,PREPARACIONES ALIMENTICIAS MEJORADORES DE PAN...,ZONA FRANCA DE BOLIVIA,CARRETERA,ZONAS FRANCAS,989,122,1549,D6,34.0,53,50,4023
2,2020.0,6.0,2.0,2208500000,GIN Y GINEBRA,COSTA RICA,AEREA,AEREA,1124,122,1551,D3,13.0,70,34,272
3,2020.0,1.0,2.0,2208500000,GIN Y GINEBRA,PERU,CARRETERA,DESAGUADERO,1124,122,1551,D3,13.0,11230,6300,42750
4,2020.0,1.0,2.0,2208500000,GIN Y GINEBRA,CHILE,CARRETERA,ARICA - CHARANA - TAMBO QUEMADO,1124,122,1551,D3,13.0,4500,2520,15480


Ejercicio: 

- Descargar el archivo con Python
- Hacer un búcle que descargue exportaciones desde el 2016 al 2020

Pista: usar un for, f-strings, ```.content``` y ```
with open(...
```

In [ ]:
# Con un loop podemos cambiar el año y descargar todas

anios = []

for a in range(2016, 2021):
  nombre = f"exp{a}.rar"
  url = f'http://web2.ine.gob.bo:8081/IneComex/BD/{nombre}'

  obj = requests.get(url)

  # guardar el contenido de obj en archivo de nombre 'nombre'

  with open(nombre, 'wb') as out:
    out.write(obj.content)
    
  print('Guardando', nombre)
  sleep(5)

Guardando exp2016.rar
Guardando exp2017.rar
Guardando exp2018.rar
Guardando exp2019.rar
Guardando exp2020.rar


## Crawling y Scraping

Podemos de un sitio inicial tomar todos los links que aparecen, y entrar para extraer la información de cada uno. 

Ejercicio:

Extraer todos los links de http://www.sice.oas.org .

- Usar ChromeDevTools para ver dónde se encuentran los datos 
- Usar requests y BeautifulSoup
- Filtrar los links que no sean nulos, contengan "/Trade/" y terminen con .asp


En una aplicación real, uno tiene que considerar como se diseña la arquitectura del servicio. Se podrían tener procesos "trabajadores" o *Workers* que hacen rastreo de nuevos links (*crawling*) alimentando una cola o (*Queue*), mientras otros que se ocupan de ir tomando de estos links y extraer los datos (*Scraping*).

[Scrapy](https://scrapy.org/) es una librería que nos ofrece abstracciones para fácilmente produccionalizar (*deploy*) productos escalables de crawling y scraping.

In [ ]:
from bs4 import BeautifulSoup

def codigo_html(url):
    '''recibe una URL y devuelve el .text de la Response'''
    headers = {'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.105 Safari/537.36'}
    return requests.get(url, headers=headers).text

In [ ]:
url = 'http://www.sice.oas.org/agreements_s.asp'

In [ ]:
# Extraer los links con BS, Regex o XPATH
soup = BeautifulSoup(..., 'html.parser')

In [ ]:
# Alternativa con Expresiones Regulares

import re
r = requests.get(url)

patron = ...

# Buscamos todo lo que coincida con nuestro patrón
subs = re.findall(patron, r.text)
subs[:10]

['/Trade/PAC_ALL/Index_Pacific_Alliance_s.asp',
 '/Trade/BOL_MEX_66/BOL_MEX_Ind_s.asp',
 '/Trade/CAFTA/CAFTADR/CAFTADRin_s.asp',
 '/Trade/CAN_EFTA_FTA/canefta_in.asp',
 '/Trade/chican_s/CAN_CHL_Index_2019_s.asp',
 '/Trade/CAN_KOR/English/CAN_KOR_index_e.asp',
 '/Trade/cancr/Spanish/indice.asp',
 '/Trade/can-isr/can-isr.asp',
 '/Trade/can-isr/CAN_ISR_2018_index_e.asp',
 '/Trade/CAN_JOR_FTA/index_e.asp']

## Cookies WTO

Vamos a buscar nuevos acuerdos comerciales en el sitio de la World Trade Organization:

https://rtais.wto.org/ 

Los acuerdos están indexados por ID, por ejemplo:

https://rtais.wto.org/UI/PublicShowRTAIDCard.aspx?rtaid=1093

Observen que en la URL hay un parámetro que es una identificación (ID) numérica


Mientras el usuario navega por un sitio, se acumula información sobre su actividad como pequeños archivos o strings denominados _cookies_, que sirven, por ejemplo, para mantener una sesión iniciada.

La clase Session de requests permite almacenar cookies

In [ ]:
from IPython.display import HTML

In [155]:
import requests 

id = 1093

# Creamos una nueva sesión
s = requests.Session() 

In [156]:
s.get('https://rtais.wto.org/')

<Response [200]>

In [164]:
# Al ingresar primero a este sitio, nos otorgan las cookies
r = rq.get(f'https://rtais.wto.org/UI/PublicShowRTAIDCard.aspx?rtaid={id}')

In [ ]:
r.content

Sin esos _cookies_, y los siguientes headers, el sitio no devuelve la información.

In [167]:
headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36',
    'Referer': f'https://rtais.wto.org/UI/PublicShowRTAIDCard.aspx?rtaid={id}'}

# Hacemos un GET pero usando la Sesión que instanciamos
r2 = s.get('https://rtais.wto.org/WEBCONTROL/ExportRTAIDCard.aspx', headers=headers)

In [ ]:
r2.content[:1000]

In [174]:
HTML(r2.text)

In [175]:
with open('data.xls', 'w') as out:
  out.write(r2.text)

Ejercicio:

- Extraigan los IDs de https://rtais.wto.org/ y descarguen los .xls para cada uno.

- Con la librería OS, creen una carpeta con el nombre de la fecha de hoy, y guarden los archivos ahí


Muchas veces las páginas web obtienen sus ingresos a partir del uso de usuarios tradicionales (humanos) pero no de los scrapers (máquinas). Por lo que estos no generan ganancias al sitio y encima pueden causar congestión en los servidores (Pudiendo causar incluso la rotura del sitio similar a lo que pasa con los [ataques DDOS](https://es.wikipedia.org/wiki/Ataque_de_denegaci%C3%B3n_de_servicio)).

Por esta razón los sitios webs suelen tener una pagina [/robots.txt](https://es.wikipedia.org/wiki/Est%C3%A1ndar_de_exclusi%C3%B3n_de_robots) donde especifican que tipo de scrapeo prefieren evitar.

Pueden ver, como ejemplos:

- https://www.google.com/robots.txt
- https://en.wikipedia.org/robots.txt

Ejercicio para la casa: 

Descargar PDFs de https://www.markiteconomics.com/Public/Release/PressReleases


Ejemplo con curl2python: identificamos el paquete, y con la herramienta armamos el paquete de requests:

Datos de la OECD https://stats.oecd.org/Index.aspx?DataSetCode=ULC_EEQ%20

In [ ]:
response.content[:1000]

In [ ]:
df = pd.read_csv('./unit_labour_costs_and_labour_productivity_(employment_based).csv')

In [ ]:
df

,LOCATION,Country,SUBJECT,Subject,MEASURE,Measure,FREQUENCY,Frequency,TIME,Time,Unit Code,Unit,PowerCode Code,PowerCode,Reference Period Code,Reference Period,Value,Flag Codes,Flags
0,AUS,Australia,ULQELP01,GDP per person employed,IXOB,Index,Q,Quarterly,2010-Q1,Q1-2010,IDX,Index,0,Units,2015_100,2015=100,89.491630,NaN,NaN
1,AUS,Australia,ULQELP01,GDP per person employed,IXOB,Index,Q,Quarterly,2010-Q2,Q2-2010,IDX,Index,0,Units,2015_100,2015=100,93.900680,NaN,NaN
2,AUS,Australia,ULQELP01,GDP per person employed,IXOB,Index,Q,Quarterly,2010-Q3,Q3-2010,IDX,Index,0,Units,2015_100,2015=100,93.225730,NaN,NaN
3,AUS,Australia,ULQELP01,GDP per person employed,IXOB,Index,Q,Quarterly,2010-Q4,Q4-2010,IDX,Index,0,Units,2015_100,2015=100,96.757190,NaN,NaN
4,AUS,Australia,ULQELP01,GDP per person employed,IXOB,Index,Q,Quarterly,2011-Q1,Q1-2011,IDX,Index,0,Units,2015_100,2015=100,89.183420,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6521,TUR,Turkey,ULQELP01,GDP per person employed,GYSA,Quarterly change on the same quarter of the pr...,Q,Quarterly,2019-Q4,Q4-2019,PC,Percentage,0,Units,NaN,NaN,6.780829,NaN,NaN
6522,TUR,Turkey,ULQELP01,GDP per person employed,GYSA,Quarterly change on the same quarter of the pr...,Q,Quarterly,2020-Q1,Q1-2020,PC,Percentage,0,Units,NaN,NaN,6.607855,NaN,NaN
6523,TUR,Turkey,ULQELP01,GDP per person employed,GYSA,Quarterly change on the same quarter of the pr...,Q,Quarterly,2020-Q2,Q2-2020,PC,Percentage,0,Units,NaN,NaN,-0.717307,NaN,NaN
6524,TUR,Turkey,ULQELP01,GDP per person employed,GYSA,Quarterly change on the same quarter of the pr...,Q,Quarterly,2020-Q3,Q3-2020,PC,Percentage,0,Units,NaN,NaN,9.567170,NaN,NaN


## Recursos extra:

### Scheduling

Scheduling es configurar una ejecución automáticamente en el tiempo. 

- [Crontab para Linux o Mac](https://tecadmin.net/crontab-in-linux-with-20-examples-of-cron-schedule/)
- [Schedule en Windows](https://stackoverflow.com/questions/132971/what-is-the-windows-version-of-cron)

### Recursos útiles
- [Tutorial de Indian Pythonista sobre APIs ocultas](https://www.youtube.com/watch?v=twuhocLtGCg)

### Herramientas útiles
- [curl2python](https://curl.trillworks.com/)
- [Visualizador de JSONs](http://jsonviewer.stack.hu/)

